# Concordance Analysis: Positive SPECT/CT + Sestamibi Scans

### Objective

Assess **localization concordance with surgical findings** for patients with **positive SPECT/CT + Sestamibi scans**.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging results and surgical outcomes)

### Outputs

- **Total scans with localization:** 285
- **Scans matching surgical side or exact gland:** 270 (**94.7%**)
---

In [65]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_cleaned_with_hyperplasia_concordance.xlsx')

In [85]:

# Define concordance function
def check_concordance(row):
    imaging = row['SPECT/CT-Sestamibi']
    surgical = row['Surgical Findings']

    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Clean combined imaging string
    imaging_clean = imaging.replace('SPECT/CT:', '').replace('Sestamibi:', '').replace('|', ',').strip()
    
    # Exclude "No SPECT/CT or Sestamibi Recorded" rows by returning Unknown here (they will be filtered out)
    if imaging_clean == 'No SPECT/CT or Sestamibi Recorded':
        return 'Unknown'


    # Treat hyperplasia as Incorrect Localization always
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    if imaging_clean in ['No Scan', 'Non-localizing']:
        return 'Non-localizing'

    imaging_glands = [g.strip().lower() for g in imaging_clean.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]

    # Exact match if any gland matches
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'

    # Side match if laterality overlaps
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'

    return 'Incorrect Localization'


In [86]:

# Apply to dataframe
preop_imaging_df['SPECT/CT-Sestamibi Calculated Concordance'] = preop_imaging_df.apply(check_concordance, axis=1)

# Filter only localized scans (exclude Non-localizing and No Scan)
localized_df = preop_imaging_df[
    ~preop_imaging_df['SPECT/CT-Sestamibi'].isin(['No Scan', 'Non-localizing'])
]

# Calculate counts
concordance_col = 'SPECT/CT-Sestamibi Calculated Concordance'
total = localized_df.shape[0]
exact = (localized_df[concordance_col] == 'Yes (Exact)').sum()
side = (localized_df[concordance_col] == 'Yes (Side)').sum()
incorrect = (localized_df[concordance_col] == 'Incorrect Localization').sum()



In [87]:
# Create summary table
summary_df = pd.DataFrame({
    'Match Type': ['Exact Match', 'Side Match', 'Incorrect Localization'],
    'Count': [exact, side, incorrect],
    'Percentage': [f"{(c/total)*100:.2f}%" if total > 0 else 'N/A' for c in [exact, side, incorrect]]
})

print(f" Total localized SPECT/CT + Sestamibi scans analyzed: {total}")
display(summary_df)

 Total localized SPECT/CT + Sestamibi scans analyzed: 299


,Match Type,Count,Percentage
0,Exact Match,134,44.82%
1,Side Match,136,45.48%
2,Incorrect Localization,16,5.35%


In [88]:
# ✅ Filter and display Exact Matches
exact_df = localized_df[
    localized_df['SPECT/CT-Sestamibi Calculated Concordance'] == 'Yes (Exact)'
][['Patient ID', 'Surgical Findings', 'SPECT/CT-Sestamibi']]
print(f"✅ Exact Matches ({exact_df.shape[0]} rows):")
display(exact_df)

# ✅ Filter and display Side Matches
side_df = localized_df[
    localized_df['SPECT/CT-Sestamibi Calculated Concordance'] == 'Yes (Side)'
][['Patient ID', 'Surgical Findings', 'SPECT/CT-Sestamibi']]
print(f"✅ Side Matches ({side_df.shape[0]} rows):")
display(side_df)

# ✅ Filter and display Incorrect Localizations
incorrect_df = localized_df[
    localized_df['SPECT/CT-Sestamibi Calculated Concordance'] == 'Incorrect Localization'
][['Patient ID', 'Surgical Findings', 'SPECT/CT-Sestamibi']]
print(f"✅ Incorrect Localizations ({incorrect_df.shape[0]} rows):")
display(incorrect_df)

✅ Exact Matches (134 rows):


,Patient ID,Surgical Findings,SPECT/CT-Sestamibi
0,156895,Right Inferior,Right Inferior
2,150593,Left Inferior,Left Inferior
6,183952,Left Superior,Left Superior
7,182519,Right Inferior,Right Inferior
8,183428,Right Inferior,"Left Inferior, Right Inferior"
...,...,...,...
391,308101,Left Inferior,Left Inferior
392,310462,Left Superior,"Left Superior, Right Inferior"
401,311569,Right Inferior,Right Inferior
402,309738,Right Inferior,Right Inferior


✅ Side Matches (136 rows):


,Patient ID,Surgical Findings,SPECT/CT-Sestamibi
3,183346,"Left Superior, Right Superior",Left Inferior
4,183255,Left Superior,Left
5,183639,Left Superior,Left Inferior
10,183675,Right Superior,Right
13,187780,Right Superior,Right Inferior
...,...,...,...
396,312172,Right Superior,Right Inferior
397,309238,Right Inferior,Right
399,305515,Right Superior,Right
400,310063,"Left Superior, Right Superior","Left, Right"


✅ Incorrect Localizations (16 rows):


,Patient ID,Surgical Findings,SPECT/CT-Sestamibi
27,151093,4 Gland Hyperplasia,Right Inferior
28,199746,4 Gland Hyperplasia,Left Inferior
185,269906,"Right Superior, Right Inferior",Midline
195,271185,Left Inferior,Right Superior
205,273483,4 Gland Hyperplasia,Left
249,167250,Inferior Side Unknown,Left Inferior
292,173307,Right Inferior,Left Inferior
298,290966,"No Tumor Identified, Total Thyroid Removed",Left
319,290927,Left Inferior,Right
325,292510,Left Inferior,Right
